In [1]:
import ccxt #pip from https://github.com/ccxt/ccxt
import pandas as pd
import numpy as np
from functools import reduce
import time, os, fnmatch, shutil
import json
import requests

import logging

In [6]:
def getaskprice(exchange,df_name):
    df_name = pd.DataFrame()
    df_name ['ask'] = exchange.iloc[0,:]
    df_name ['Pairs'] = exchange.iloc[0,:].index
    return df_name.reset_index(drop = True)

def getbidprice(exchange,df_name):
    df_name = pd.DataFrame()
    df_name ['Prices'] = exchange.iloc[4,:]
    df_name ['Pairs'] = exchange.iloc[4,:].index
    return df_name.reset_index(drop = True)

In [7]:
binance = pd.DataFrame.from_dict(ccxt.binance().fetch_tickers(),orient = 'columns')
bittrex = pd.DataFrame.from_dict(ccxt.bittrex().fetch_tickers(),orient = 'columns')
bitfinex = pd.DataFrame.from_dict(ccxt.bitfinex().fetch_tickers(),orient = 'columns')
quoinex = pd.DataFrame.from_dict(ccxt.quoinex().fetch_tickers(),orient = 'columns')
kraken = pd.DataFrame.from_dict(ccxt.kraken().fetch_tickers(),orient = 'columns')
kucoin = pd.DataFrame.from_dict(ccxt.kucoin().fetch_tickers(),orient = 'columns')
df_binance = pd.DataFrame()
df_bittrex = pd.DataFrame()
df_bitfinex = pd.DataFrame()
df_quoinex = pd.DataFrame()
df_kraken = pd.DataFrame()
df_kucoin = pd.DataFrame()

In [8]:
binance_bid = getbidprice(binance,df_binance)
bittrex_bid = getbidprice(bittrex,df_bittrex)
quoinex_bid = getbidprice(quoinex,df_quoinex)
bitfinex_bid = getbidprice(bitfinex,df_bitfinex)
kraken_bid = getbidprice(kraken,df_kraken)
kucoin_bid = getbidprice(kucoin,df_kucoin)
binance_ask = getaskprice(binance,df_binance)
bittrex_ask = getaskprice(bittrex,df_bittrex)
quoinex_ask = getaskprice(quoinex,df_quoinex)
bitfinex_ask = getaskprice(bitfinex,df_bitfinex)
kraken_ask = getaskprice(kraken,df_kraken)
kucoin_ask = getaskprice(kucoin,df_kucoin)

In [9]:
dfs = [binance_bid,bittrex_bid,bitfinex_bid,quoinex_bid,kraken_bid, kucoin_bid,  
       binance_ask,bittrex_ask,bitfinex_ask,quoinex_ask,kraken_ask, kucoin_ask, ]
df_final = reduce(lambda left,right: pd.merge(left,right,how="outer",on='Pairs'), dfs)
df_final.columns = ['Binance bid','Pairs','Bittrex bid','Bitfinex bid','Quoinex bid',
                    'Kraken bid','Kucoin bid','Binance ask', 'Bittrex ask','Bitfinex ask',
                    'Quoinex ask','Kraken ask','Kucoin ask']
df_final.fillna(np.nan,inplace=True)

In [10]:
df_final ['Max bid'] = df_final[['Binance bid','Bittrex bid','Bitfinex bid','Quoinex bid','Kraken bid','Kucoin bid']].max(axis=1)
df_final ['Min ask'] = df_final[['Binance ask','Bittrex ask','Bitfinex ask','Quoinex ask','Kraken ask','Kucoin ask']].min(axis=1)
df_final ['%'] = (df_final['Max bid']/df_final['Min ask']*100 - 100)
df_final.sort_values(by='%', ascending = False).head(40)

,Binance bid,Pairs,Bittrex bid,Bitfinex bid,Quoinex bid,Kraken bid,Kucoin bid,Binance ask,Bittrex ask,Bitfinex ask,Quoinex ask,Kraken ask,Kucoin ask,Max bid,Min ask,%
934,NaN,FLP/QASH,NaN,NaN,1.945193e-02,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1.945193e-02,0.000000e+00,inf
421,1.555800e-02,ZEC/BTC,0.015505,0.015600,1.202000e-02,0.015550,NaN,1.557000e-02,0.015613,0.015603,0.000000e+00,0.015610,NaN,1.560000e-02,0.000000e+00,inf
38,0.000000e+00,BCH/BTC,0.042715,NaN,4.264264e-02,0.042760,NaN,0.000000e+00,0.042960,NaN,4.298185e-02,0.042890,NaN,4.276000e-02,0.000000e+00,inf
39,0.000000e+00,BCH/ETH,1.093197,NaN,NaN,NaN,NaN,0.000000e+00,1.108949,NaN,NaN,NaN,NaN,1.093197e+00,0.000000e+00,inf
40,0.000000e+00,BCH/USDT,161.855651,NaN,NaN,NaN,NaN,0.000000e+00,163.507181,NaN,NaN,NaN,NaN,1.618557e+02,0.000000e+00,inf
47,7.000000e-06,BCN/ETH,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,7.000000e-06,0.000000e+00,inf
1006,NaN,MT/ETH,NaN,NaN,3.110000e-06,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,3.110000e-06,0.000000e+00,inf
1019,NaN,PAL/ETH,NaN,NaN,2.400000e-07,NaN,0.000036,NaN,NaN,NaN,0.000000e+00,NaN,0.000038,3.620000e-05,0.000000e+00,inf
463,NaN,CMCT/BTC,0.000000,NaN,3.200000e-07,NaN,NaN,NaN,0.000000,NaN,9.700000e-07,NaN,NaN,3.200000e-07,0.000000e+00,inf
1020,NaN,PAL/QASH,NaN,NaN,4.499900e-04,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,4.499900e-04,0.000000e+00,inf


In [11]:
def returnbidcolumn(df):
    bid = df['Max bid']
    zz = (df.iloc[:7] == bid)
    name = df_final.iloc[:,:7].columns[zz].values[0]
    return name


def returnaskcolumn(df):
    ask = df['Min ask']
    zz = (df.iloc[7:] == ask)
    name = df_final.iloc[:,7:].columns[zz].values[0]
    return name

df_final ['Sell at'] = df_final.apply(returnbidcolumn, axis = 1)
df_final ['Buy at'] = df_final.apply(returnaskcolumn, axis = 1)

IndexError: ('index 0 is out of bounds for axis 0 with size 0', 'occurred at index 998')

In [12]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)
model = df_final.sort_values(by='%', ascending = False) [:50]
model.to_csv("Prices" + timestamp +'.csv')